In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import warnings

In [3]:
def Print_values(observed):

    res = scipy.stats.chi2_contingency(observed=observed) 
    chi2 = res.statistic
    pvalue = res.pvalue
    expected = pd.DataFrame(res.expected_freq, columns=['Handbagage', 'Bodyscan', 'Douane'])   

    print("\n", 100*"-", "\n")
    print("\nH0: There is no relationship between the variables.\n")

    print("\n", 100*"-", "\n")
    print("Chi^2: %.3f \nP-value: %.3g\n" %(chi2, pvalue))

    if pvalue <= 0.5:
        print("Null Hypothesis rejected: The test results detect a significant relationship between the variables.")
    
    else:
        print("Null Hypothesis accepted: The test results don't detect a significant relationship between the variables.")
    print("\n", 100*"-", "\n")

    return expected

In [4]:
def Normal_test(x):
    k2, p = scipy.stats.normaltest(x)

    print("\n", 100*"-", "\n")
    print("\nH0: The results follow a normal distribution.\n")

    print("\n", 100*"-", "\n")
    print("k^2: %.3f \nP-value: %.3g\n" %(k2, p))

    if p <= 0.5:
        print("Null Hypothesis rejected: The test results don't detect a significant relationship with the normal distribution.")
    
    else:
        print("Null Hypothesis accepted: The test results detect a significant relationship with the normal distribution.")
    print("\n", 100*"-", "\n")

In [10]:
df = pd.read_excel("Data.xlsx", header=1)
df = df.loc[:, ~df.columns.str.contains("^metingnr")]
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df = df.rename(columns={"doorlooptijd [s]":"Handbage", "doorlooptijd [s].1":"Bodyscan", "doorlooptijd [s].2":"Douane"})
df


,Handbage,Bodyscan,Douane
0,200,215,218
1,207,174,183
2,191,210,210
3,215,178,225
4,220,162,235
5,205,188,179
6,225,205,230
7,157,220,267
8,220,167,201
9,225,184,196


In [ ]:
observed = df.iloc[0:9, 0:3]
expected = Print_values(observed)



 ---------------------------------------------------------------------------------------------------- 


H0: There is no relationship between the variables.


 ---------------------------------------------------------------------------------------------------- 

Chi^2: 50.423 
P-value: 1.96e-05

Null Hypothesis rejected: The test results detect a significant relationship between the variables.

 ---------------------------------------------------------------------------------------------------- 



In [ ]:
expected

,Handbagage,Bodyscan,Douane
0,211.498093,197.589795,223.912112
1,188.443799,176.051571,199.504630
2,204.147449,190.722535,216.130016
3,206.486290,192.907572,218.606138
4,206.152170,192.595424,218.252406
5,191.116760,178.548756,202.334483
6,220.519339,206.017796,233.462865
7,215.173416,201.023425,227.803160
8,196.462684,183.543127,207.994189


In [ ]:
x = df['Handbage'].values

Normal_test(x)


 ---------------------------------------------------------------------------------------------------- 


H0: The results follow a normal distribution.


 ---------------------------------------------------------------------------------------------------- 

k^2: 9.393 
P-value: 0.00913

Null Hypothesis rejected: The test results don't detect a significant relationship with the normal distribution.

 ---------------------------------------------------------------------------------------------------- 



c:\Users\lteeg\Documents\Github\School\python\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\lteeg\AppData\Local\Temp\ipykernel_49732\1587176214.py:18: UserWarning: user
  if warnings.warn("user", UserWarning):
